<a href="https://colab.research.google.com/github/EddyGiusepe/Repasso_Python/blob/main/salvando_TENSORFLOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h2 align="center">Usando Google Colab</h2>


Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro


Este Script foi baseado no [Machine Learning Mastery - Jason Brownlee PhD](https://machinelearningmastery.com/google-colab-for-machine-learning-projects/)

# Exemplo: salvar o progresso do modelo no Google Drive

O ``Google Colab`` é provavelmente a maneira mais fácil de nos fornecer recursos avançados de ``GPU`` para seu projeto de aprendizado de máquina. Mas na versão gratuita do Colab, o Google limita o tempo que podemos usar nosso notebook Colab em cada sessão. Nosso kernel pode terminar sem motivo. Podemos reiniciar nosso notebook e continuar nosso trabalho, mas podemos perder tudo na memória. Isso é um problema se precisarmos treinar nosso modelo por muito tempo. Nossa instância do Colab pode ser encerrada antes que o treinamento seja concluído.

Usando a extensão do Google Colab para montar nosso retorno de chamada do ``Google Drive`` e ``Keras ModelCheckpoint``, podemos salvar o progresso do nosso modelo no Google Drive. Isso é particularmente útil para contornar os tempos limite do Colab. É mais tolerante para usuários pagos do ``Pro`` e ``Pro+``, mas sempre há uma chance de que nosso treinamento de modelo termine no meio do caminho em momentos aleatórios. É valioso se não quisermos perder nosso modelo parcialmente treinado.

Para esta demonstração, usaremos o modelo ``LeNet-5`` no conjunto de dados ``MNIST``.

In [1]:
# Importamos algumas bibliotecas necessárias


import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPool2D
from keras.models import Model



In [2]:
# Criando uma classe

class LeNet5(tf.keras.Model):

  def __init__(self):
    super(LeNet5, self).__init__()
    
    #creating layers in initializer
    self.conv1 = Conv2D(filters=6, kernel_size=(5,5), padding="same", activation="relu")
    self.max_pool2x2 = MaxPool2D(pool_size=(2,2))
    self.conv2 = Conv2D(filters=16, kernel_size=(5,5), padding="same", activation="relu")
    self.flatten = Flatten()
    self.fc1 = Dense(units=120, activation="relu")
    self.fc2 = Dense(units=84, activation="relu")
    self.fc3=Dense(units=10, activation="softmax")
  
  def call(self, input_tensor):
    conv1 = self.conv1(input_tensor)
    maxpool1 = self.max_pool2x2(conv1)
    conv2 = self.conv2(maxpool1)
    maxpool2 = self.max_pool2x2(conv2)
    flatten = self.flatten(maxpool2)
    fc1 = self.fc1(flatten)
    fc2 = self.fc2(fc1)
    fc3 = self.fc3(fc2)
    return fc3

Em seguida, para ``salvar o progresso do modelo durante o treinamento no Google Drive``, primeiro precisamos montar nosso Google Drive em nosso ambiente Colab.

In [3]:
# Instanciamos nosso Google Drive

import os
from google.colab import drive


MOUNTPOINT = "/content/gdrive"
DATADIR = os.path.join(MOUNTPOINT, "MyDrive")
drive.mount(MOUNTPOINT)

Mounted at /content/gdrive


Depois, declaramos o Callback para salvar nosso modelo de checkpoint no Google Drive.

In [4]:
# Observarás que será criada uma pasta com o nome: checkpoints
 
checkpoint_path = DATADIR + "/checkpoints/cp-epoch-{epoch}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

Em seguida, começamos o treinamento no conjunto de dados ``MNIST`` com os retornos de chamada do ponto de verificação para garantir que possamos retomar na última época, caso a sessão do Colab expire:

In [5]:
# Baixamos os Dados
mnist_digits = keras.datasets.mnist

# Separamos em Dados de Treinamento e Dados de teste
(train_images, train_labels), (test_images, test_labels) = mnist_digits.load_data()
 
input_layer = Input(shape=(28,28,1))

# Instanciamos nosso Modelo
model = LeNet5()(input_layer)
model = Model(inputs=input_layer, outputs=model)

# Compilamos nosso Modelo
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics="acc")

# Fit
model.fit(x=train_images, y=train_labels, batch_size=256,
          validation_data = [test_images, test_labels], epochs=5, callbacks=[cp_callback])

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/5
235/235 [==============================] - ETA: 0s - loss: 1.3052 - acc: 0.8325
Epoch 1: saving model to /content/gdrive/MyDrive/checkpoints/cp-epoch-1.ckpt
235/235 [==============================] - 42s 174ms/step - loss: 1.3052 - acc: 0.8325 - val_loss: 0.1932 - val_acc: 0.9456
Epoch 2/5
235/235 [==============================] - ETA: 0s - loss: 0.1503 - acc: 0.9563
Epoch 2: saving model to /content/gdrive/MyDrive/checkpoints/cp-epoch-2.ckpt
235/235 [==============================] - 41s 173ms/step - loss: 0.1503 - acc: 0.9563 - val_loss: 0.1059 - val_acc: 0.9684
Epoch 3/5
235/235 [==============================] - ETA: 0s - loss: 0.0909 - acc: 0.9725
Epoch 3: saving model to /content/gdrive/MyDrive/checkpoints/cp-epoch-3.ckpt
235/235 [==============================] - 41s 173ms/step - loss: 0.0909 - acc: 0.9725 - val_loss: 0.0849 - val_acc: 0.9756
Epoch 4/5
235/235 [==============================] - ETA: 0s -

E a partir da saída, podemos ver que os pontos de verificação foram salvos. Olhando para minha pasta do Google Drive, também podemos ver os pontos de verificação armazenados lá.


A instância do Colab está no ambiente de nuvem do Google. A máquina que está rodando tem algum armazenamento, então podemos instalar um pacote ou baixar alguns arquivos nele. No entanto, não devemos salvar nosso ponto de verificação lá porque não temos garantia de que o recuperaremos depois que nossa sessão for encerrada. Portanto, acima, montamos nosso Google Drive na instância e salvamos o checkpoint em nosso Google Drive. É assim que podemos ter certeza de que os arquivos do ponto de verificação estão acessíveis.